In [1]:
import os
import sys
import asyncio
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)


In [2]:
from python.agents.analyst_agent import analyst_agent

In [3]:
session_service = InMemorySessionService()
APP_NAME = "test_app"
USER_ID = "user_1"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

session = session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

# --- Runner ---
# Key Concept: Runner orchestrates the agent execution loop.
runner = Runner(
    agent=analyst_agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)
print(f"Runner created for agent '{runner.agent.name}'.")

Session created: App='test_app', User='user_1', Session='session_001'
Runner created for agent 'analyst_agent'.


In [4]:
def call_agent(query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)
    
    for event in events:
        if event.get_function_calls:
            for call in event.get_function_calls():
                print(f"  Tool: {call.name}, Args: {call.args}")
        if event.is_final_response():
            return event.content.parts[0].text

In [5]:
query = "วิเคราะห์หุ้นปตท"
r1 = call_agent(query)

  Tool: transfer_to_agent, Args: {'agent_name': 'search_agent'}
  Tool: fetch_financial_data, Args: {'symbol': 'PTT'}
  Tool: transfer_to_agent, Args: {'agent_name': 'analyst_agent'}


In [6]:
print(f"Response: {r1}")

Response: จากการวิเคราะห์งบการเงินของ ปตท. (PTT) ในช่วง 5 ปีที่ผ่านมา (2563-2567):

**ภาพรวมผลประกอบการ:**

*   **รายได้:** มีความผันผวนตามราคาน้ำมันและสภาวะเศรษฐกิจโลก โดยรายได้เติบโตขึ้นอย่างมากในปี 2564 และ 2565 ก่อนจะปรับตัวลดลงเล็กน้อยในปี 2566 และ 2567
*   **กำไรสุทธิ:** มีทิศทางสอดคล้องกับรายได้ โดยกำไรสุทธิเพิ่มขึ้นอย่างโดดเด่นในปี 2564 และ 2565 แต่ลดลงในปี 2566 และ 2567 สาเหตุหลักมาจากความผันผวนของราคาพลังงานและต้นทุนการผลิตที่สูงขึ้น
*   **กระแสเงินสด:** กระแสเงินสดจากการดำเนินงานค่อนข้างแข็งแกร่ง แต่กระแสเงินสดอิสระ (Free Cash Flow) มีความผันผวน ซึ่งเป็นผลมาจากการลงทุนในโครงการต่างๆ

**ฐานะทางการเงิน:**

*   **สินทรัพย์:** สินทรัพย์รวมของ ปตท. เพิ่มขึ้นอย่างต่อเนื่องในช่วง 5 ปีที่ผ่านมา สะท้อนถึงการขยายตัวของธุรกิจ
*   **หนี้สิน:** หนี้สินรวมเพิ่มขึ้นเช่นกัน โดยมีสัดส่วนหนี้สินต่อส่วนของผู้ถือหุ้น (Debt-to-Equity Ratio) ค่อนข้างสูง แต่ยังอยู่ในระดับที่บริหารจัดการได้
*   **สภาพคล่อง:** สภาพคล่องของ ปตท. ยังอยู่ในเกณฑ์ดี โดยมีสินทรัพย์หมุนเวียนมากกว่าหนี้สินหมุนเวียน

**แนวโน

In [7]:
query = "รายได้ของ SCB เพิ่มขึ้นหรือเปล่า ละเอียด"
r2 = call_agent(query)

  Tool: transfer_to_agent, Args: {'agent_name': 'search_agent'}
  Tool: fetch_financial_data, Args: {'symbol': 'SCB'}
  Tool: transfer_to_agent, Args: {'agent_name': 'analyst_agent'}


In [8]:
print(f"Response: {r2}")

Response: จากข้อมูลที่ได้รับ SCB มีรายได้เพิ่มขึ้นในช่วง 5 ปีที่ผ่านมา แต่มีรายละเอียดที่น่าสนใจดังนี้:

*   **ปี 2563:** รายได้รวม 98,118 ล้านบาท
*   **ปี 2564:** รายได้รวม 108,318 ล้านบาท (เพิ่มขึ้น 10.39% YoY)
*   **ปี 2565:** รายได้รวม 120,592 ล้านบาท (เพิ่มขึ้น 11.33% YoY)
*   **ปี 2566:** รายได้รวม 127,503 ล้านบาท (เพิ่มขึ้น 5.73% YoY)
*   **ปี 2567:** รายได้รวม 129,779 ล้านบาท (เพิ่มขึ้น 1.79% YoY)
*   **ไตรมาส 1 ปี 2568 (TTM):** รายได้รวม 130,412 ล้านบาท (เพิ่มขึ้น 0.84% YoY)

**สรุป:**

SCB มีแนวโน้มรายได้เพิ่มขึ้นอย่างต่อเนื่องตั้งแต่ปี 2563 ถึงปี 2567 อย่างไรก็ตาม อัตราการเติบโตของรายได้เริ่มชะลอตัวลงในปี 2566 และ 2567 และยังคงเติบโตในอัตราที่ต่ำในไตรมาส 1 ปี 2568 เมื่อเทียบกับช่วงเดียวกันของปีก่อนหน้า

**รายละเอียดเพิ่มเติม:**

*   การเติบโตของรายได้ส่วนใหญ่มาจากรายได้ดอกเบี้ยสุทธิ (Net Interest Income) ที่เพิ่มขึ้น
*   อย่างไรก็ตาม รายได้ที่ไม่ใช่ดอกเบี้ย (Non-Interest Income) มีความผันผวน และมีแนวโน้มลดลงเล็กน้อยในปีล่าสุด
*   SCB ยังคงต้องเผชิญกับความท้าทายในการควบคุมค่า

In [9]:
query = "วิเคราะห์หุ้นบางจาก"
r3 = call_agent(query)

  Tool: transfer_to_agent, Args: {'agent_name': 'search_agent'}
  Tool: fetch_financial_data, Args: {'symbol': 'BCP'}
  Tool: transfer_to_agent, Args: {'agent_name': 'analyst_agent'}


In [10]:
print(f"Response: {r3}")

Response: จากการวิเคราะห์งบการเงินของ บมจ.บางจาก คอร์ปอเรชั่น (BCP) ในช่วง 5 ปีที่ผ่านมา (พ.ศ. 2563-2567):

**ภาพรวมผลประกอบการ:**

*   **รายได้:** มีการเติบโตอย่างมีนัยสำคัญ โดยเฉพาะในปี 2565 และ 2567 ซึ่งมีอัตราการเติบโตของรายได้สูงถึง 64.49% และ 52.88% ตามลำดับ แสดงถึงการฟื้นตัวของธุรกิจหลังสถานการณ์โควิด-19 และการขยายตัวของธุรกิจ
*   **กำไรสุทธิ:** แม้ว่ารายได้จะเติบโต แต่กำไรสุทธิกลับมีความผันผวนอย่างมาก โดยในปี 2567 กำไรสุทธิลดลงถึง 83.49% เมื่อเทียบกับปีก่อนหน้า สาเหตุหลักมาจากต้นทุนการผลิตที่สูงขึ้น และค่าใช้จ่ายในการดำเนินงานที่เพิ่มขึ้น
*   **กระแสเงินสด:** กระแสเงินสดจากการดำเนินงานค่อนข้างผันผวน แต่โดยรวมยังคงเป็นบวก อย่างไรก็ตาม กระแสเงินสดอิสระ (Free Cash Flow) ก็มีความผันผวนเช่นกัน ซึ่งเป็นผลมาจากการลงทุนในโครงการต่างๆ และการเปลี่ยนแปลงของเงินทุนหมุนเวียน

**ฐานะทางการเงิน:**

*   **สินทรัพย์:** สินทรัพย์รวมของ BCP เพิ่มขึ้นอย่างต่อเนื่องในช่วง 5 ปีที่ผ่านมา สะท้อนถึงการขยายตัวของธุรกิจ
*   **หนี้สิน:** หนี้สินรวมเพิ่มขึ้นเช่นกัน โดยมีสัดส่วนหนี้สินต่อส่วนของผู้ถือหุ้น (

In [11]:
query = "สภาพคล่องการเงินหุ้นทรูเป็นยังไง"
r4 = call_agent(query)

  Tool: transfer_to_agent, Args: {'agent_name': 'search_agent'}
  Tool: fetch_financial_data, Args: {'symbol': 'TRUE'}
  Tool: transfer_to_agent, Args: {'agent_name': 'analyst_agent'}


In [12]:
print(f"Response: {r4}")

Response: จากการวิเคราะห์ข้อมูลทางการเงินล่าสุดของ TRUE (ข้อมูล TTM สิ้นสุดวันที่ 31 มีนาคม 2525 และงบปี 2567):

**สินทรัพย์และหนี้สินหมุนเวียน:**

*   TRUE มีสินทรัพย์หมุนเวียน (Total Current Assets) น้อยกว่าหนี้สินหมุนเวียน (Total Current Liabilities) อย่างมาก โดยมี Working Capital ติดลบถึง -117,221 ล้านบาท ในช่วง TTM และ -115,403 ล้านบาท ในปี 2567 แสดงว่าบริษัทอาจมีปัญหาในการชำระหนี้สินระยะสั้น
*   เงินสดและรายการเทียบเท่าเงินสด (Cash & Equivalents) ลดลงอย่างมาก (-51.02% YoY) ในช่วง TTM และ -20.18% ในปี 2567 บ่งชี้ถึงสภาพคล่องที่ตึงตัวขึ้น

**กระแสเงินสด:**

*   กระแสเงินสดจากการดำเนินงาน (Operating Cash Flow) ยังคงเป็นบวกที่ 77,821 ล้านบาท (TTM) และ 78,446 ล้านบาท (ปี 2567) แต่ลดลง -5.72% YoY ในช่วง TTM
*   กระแสเงินสดอิสระ (Free Cash Flow) เป็นบวกที่ 52,519 ล้านบาท (TTM) และ 52,166 ล้านบาท (ปี 2567) อย่างไรก็ตาม อัตรากำไรจากกระแสเงินสดอิสระ (Free Cash Flow Margin) ค่อนข้างต่ำที่ 25.48% (TTM)

**สรุป:**

หุ้น TRUE มีสภาพคล่องทางการเงินที่น่ากังวล เนื่องจากมีหนี้สินหมุนเวียนมากกว่าส